<details>
<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax Scaler Informer](#3-minmax-scaler-informer)
- [4. MinMax Scaler PatchTST](#4-minmax-scaler-patchtst)

</details>


Here we perform a check on **Italy** dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison_IT". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

In [27]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Standard Scaler Informer

In [28]:
cuda_device = "0"

# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/standard"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [29]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Set the best learning rate based on pred_len
            if pred_len == "24":
                lr = 0.00001
            elif pred_len in ["96", "168"]:
                lr = 0.0001

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type standard \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_rat

In [30]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_IT.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_IT.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2203  0.4694  0.2935  0.4299
              2         24        0.2171  0.4659  0.2862  0.4267
              1         96        0.3708  0.6090  0.4044  0.5576
              2         96        0.3624  0.6020  0.4008  0.5512
              1         168       0.4241  0.6512  0.4371  0.5964
              2         168       0.3813  0.6175  0.4258  0.5655
MAE           1         24        0.2123  0.4607  0.2703  0.4220
              2         24        0.2143  0.4629  0.2699  0.4240
              1         96        0.3860  0.6213  0.3768  0.5689
              2         96        0.3996  0.6322  0.3920  0.5788
              1         168       0.4348  0.6594  0.4239  0.6039
              2         168       0.4569  0.6760  0.4328  0.6191

In [31]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1680994.875  1296.5319   861.9956  0.0911
              2         24        1580196.750  1257.0587   835.5945  0.0883
              1         96        3333727.750  1825.8499  1242.7080  0.1285
              2         96        3329982.000  1824.8239  1235.0143  0.1284
              1         168       4642685.000  2154.6890  1412.2063  0.1518
              2         168       3976689.750  1994.1639  1357.4500  0.1405
MAE           1         24        1440519.500  1200.2164   759.9636  0.0843
              2         24        1422171.750  1192.5485   752.7681  0.0838
              1         96        2678728.500  1636.6821  1075.3901  0.1152
              2         96        3289923.750  1813.8147  1161.3967  0.1276
              1         168       4242708.000  2059.7834  1329.1781  0.1451
              2         168       4784611.000  2187.3755  1385.8914  0.1541

In [32]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2133  0.4618  0.2701  0.4230
         MSE            0.2187  0.4676  0.2899  0.4283
96       MAE            0.3928  0.6267  0.3844  0.5738
         MSE            0.3666  0.6055  0.4026  0.5544
168      MAE            0.4459  0.6677  0.4283  0.6115
         MSE            0.4027  0.6344  0.4314  0.5810

In [33]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.431346e+06  1196.3824   756.3658  0.0841
         MSE            1.630596e+06  1276.7953   848.7950  0.0897
96       MAE            2.984326e+06  1725.2484  1118.3934  0.1214
         MSE            3.331855e+06  1825.3369  1238.8611  0.1285
168      MAE            4.513660e+06  2123.5795  1357.5347  0.1496
         MSE            4.309687e+06  2074.4265  1384.8281  0.1461

# 2. Standard Scaler PatchTST

In [34]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.00001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type standard \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_512_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_r

	iters: 600, epoch: 2 | loss: 0.1344917
	speed: 0.0316s/iter; left time: 516.8431s
	iters: 700, epoch: 2 | loss: 0.2044084
	speed: 0.0316s/iter; left time: 513.6871s
	iters: 800, epoch: 2 | loss: 0.1776914
	speed: 0.0316s/iter; left time: 510.5148s
-------------------------------------------------------------------------------------
Epoch: 2
Cost time: 00h:00m:28.40s
Steps: 893 | Train Loss: 0.2219412 Vali Loss: 0.1973972 Test Loss: 0.2222147
Validation loss decreased (0.327203 --> 0.197397).  Saving model ...
Updating learning rate to 1e-05
	iters: 100, epoch: 3 | loss: 0.1550447
	speed: 0.1150s/iter; left time: 1837.0177s
	iters: 200, epoch: 3 | loss: 0.2049012
	speed: 0.0316s/iter; left time: 501.1151s
	iters: 300, epoch: 3 | loss: 0.1628761
	speed: 0.0316s/iter; left time: 497.9564s
	iters: 400, epoch: 3 | loss: 0.1779979
	speed: 0.0316s/iter; left time: 495.1875s
	iters: 500, epoch: 3 | loss: 0.1302713
	speed: 0.0316s/iter; left time: 491.8090s
	iters: 600, epoch: 3 | loss: 0.2445

In [35]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_IT.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2019  0.4494  0.2719  0.4116
              2         24        0.2052  0.4530  0.2725  0.4149
              1         96        0.3542  0.5951  0.3814  0.5449
              2         96        0.3525  0.5937  0.3790  0.5436
              1         168       0.3829  0.6188  0.4046  0.5667
              2         168       0.3796  0.6161  0.4080  0.5643
MAE           1         24        0.2024  0.4498  0.2565  0.4120
              2         24        0.2047  0.4525  0.2566  0.4144
              1         96        0.3682  0.6068  0.3692  0.5556
              2         96        0.3796  0.6161  0.3692  0.5641
              1         168       0.4112  0.6413  0.3928  0.5873
              2         168       0.4168  0.6456  0.3943  0.5913

In [36]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1312173.250  1145.5013   756.4719  0.0805
              2         24        1321465.125  1149.5499   759.6286  0.0808
              1         96        2694919.000  1641.6208  1102.1880  0.1155
              2         96        2625019.000  1620.1910  1093.8665  0.1140
              1         168       3139875.250  1771.9694  1196.2166  0.1248
              2         168       3260984.750  1805.8197  1226.3945  0.1272
MAE           1         24        1205955.250  1098.1599   689.5394  0.0772
              2         24        1217987.250  1103.6246   689.6762  0.0776
              1         96        2643049.500  1625.7458  1045.0140  0.1144
              2         96        2620916.500  1618.9244  1033.9302  0.1139
              1         168       3088662.000  1757.4590  1125.1515  0.1238
              2         168       3025792.000  1739.4803  1117.3407  0.1225

In [37]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_IT.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_IT.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2036  0.4512  0.2565  0.4132
         MSE            0.2036  0.4512  0.2722  0.4132
96       MAE            0.3739  0.6115  0.3692  0.5599
         MSE            0.3534  0.5944  0.3802  0.5443
168      MAE            0.4140  0.6434  0.3936  0.5893
         MSE            0.3813  0.6175  0.4063  0.5655

In [38]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.211971e+06  1100.8923   689.6078  0.0774
         MSE            1.316819e+06  1147.5256   758.0503  0.0806
96       MAE            2.631983e+06  1622.3351  1039.4721  0.1142
         MSE            2.659969e+06  1630.9059  1098.0272  0.1148
168      MAE            3.057227e+06  1748.4697  1121.2461  0.1232
         MSE            3.200430e+06  1788.8945  1211.3055  0.1260

In [39]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'standard_unscaled_IT')

# 3. MinMax Scaler Informer

We can use now "ReLU" activation function due to MinMax Scaler.

With BS 1036, ReLU - results are bad. (as twice as bad as with 32!)

In [40]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [41]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"
            # Set the best learning rate based on pred_len
            if pred_len == "24":
                lr = 0.00001
            elif pred_len in ["96", "168"]:
                lr = 0.0001

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_rate=1

	iters: 600, epoch: 2 | loss: 0.0234527
	speed: 0.0299s/iter; left time: 496.8980s
	iters: 700, epoch: 2 | loss: 0.0208072
	speed: 0.0297s/iter; left time: 491.2887s
	iters: 800, epoch: 2 | loss: 0.0212584
	speed: 0.0300s/iter; left time: 493.1181s
	iters: 900, epoch: 2 | loss: 0.0207911
	speed: 0.0305s/iter; left time: 498.3938s
-------------------------------------------------------------------------------------
Epoch: 2
Cost time: 00h:00m:27.57s
Steps: 906 | Train Loss: 0.0312752 Vali Loss: 0.0131057 Test Loss: 0.0143359
Validation loss decreased (0.084486 --> 0.013106).  Saving model ...
Updating learning rate to 1e-05
	iters: 100, epoch: 3 | loss: 0.0202638
	speed: 0.0864s/iter; left time: 1400.4847s
	iters: 200, epoch: 3 | loss: 0.0160392
	speed: 0.0298s/iter; left time: 480.2914s
	iters: 300, epoch: 3 | loss: 0.0193679
	speed: 0.0298s/iter; left time: 477.2098s
	iters: 400, epoch: 3 | loss: 0.0150699
	speed: 0.0298s/iter; left time: 474.2034s
	iters: 500, epoch: 3 | loss: 0.0208

In [42]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_IT.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_IT.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0103  0.1017  0.0627  0.3843
              2         24        0.0107  0.1036  0.0647  0.3915
              1         96        0.0189  0.1375  0.0891  0.5199
              2         96        0.0185  0.1359  0.0884  0.5138
              1         168       0.0205  0.1430  0.0950  0.5412
              2         168       0.0204  0.1430  0.0952  0.5410
MAE           1         24        0.0109  0.1046  0.0631  0.3954
              2         24        0.0107  0.1034  0.0609  0.3906
              1         96        0.0196  0.1402  0.0873  0.5300
              2         96        0.0219  0.1479  0.0906  0.5591
              1         168       0.0233  0.1525  0.0953  0.5771
              2         168       0.0232  0.1525  0.0961  0.5769

In [43]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        1547444.00  1243.9630   822.1915  0.0874
              2         24        1827726.75  1351.9344   885.4187  0.0950
              1         96        3707538.75  1925.4971  1254.8304  0.1355
              2         96        3334153.00  1825.9663  1212.4634  0.1285
              1         168       4258305.50  2063.5662  1351.6522  0.1454
              2         168       4184471.50  2045.5980  1344.2694  0.1441
MAE           1         24        1645302.50  1282.6935   826.3154  0.0901
              2         24        1408336.25  1186.7334   765.2639  0.0834
              1         96        3098940.50  1760.3807  1143.3651  0.1239
              2         96        3245477.75  1801.5210  1160.1573  0.1268
              1         168       4006630.25  2001.6569  1286.0061  0.1410
              2         168       4571669.00  2138.1462  1343.2222  0.1506

In [44]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_IT.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_IT.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0108  0.1040  0.0620  0.3930
         MSE            0.0105  0.1026  0.0637  0.3879
96       MAE            0.0208  0.1440  0.0890  0.5446
         MSE            0.0187  0.1367  0.0888  0.5169
168      MAE            0.0233  0.1525  0.0957  0.5770
         MSE            0.0205  0.1430  0.0951  0.5411

In [45]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                           
24       MAE            1526819.375  1234.7134   795.7897  0.0868
         MSE            1687585.375  1297.9487   853.8051  0.0912
96       MAE            3172209.125  1780.9509  1151.7612  0.1253
         MSE            3520845.875  1875.7317  1233.6469  0.1320
168      MAE            4289149.625  2069.9016  1314.6141  0.1458
         MSE            4221388.500  2054.5821  1347.9608  0.1447

# 4. MinMax Scaler PatchTST

In [46]:
log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [47]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.00001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_512_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_rate

Steps: 893 | Train Loss: 0.0263476 Vali Loss: 0.0163548 Test Loss: 0.0173455
Validation loss decreased (inf --> 0.016355).  Saving model ...
Updating learning rate to 1e-05
	iters: 100, epoch: 2 | loss: 0.0123138
	speed: 0.1168s/iter; left time: 1969.3740s
	iters: 200, epoch: 2 | loss: 0.0103427
	speed: 0.0317s/iter; left time: 531.9735s
	iters: 300, epoch: 2 | loss: 0.0163344
	speed: 0.0317s/iter; left time: 529.0671s
	iters: 400, epoch: 2 | loss: 0.0124028
	speed: 0.0317s/iter; left time: 525.5516s
	iters: 500, epoch: 2 | loss: 0.0095551
	speed: 0.0318s/iter; left time: 523.4546s
	iters: 600, epoch: 2 | loss: 0.0066554
	speed: 0.0319s/iter; left time: 522.9389s
	iters: 700, epoch: 2 | loss: 0.0103693
	speed: 0.0320s/iter; left time: 520.0860s
	iters: 800, epoch: 2 | loss: 0.0089413
	speed: 0.0319s/iter; left time: 516.5414s
-------------------------------------------------------------------------------------
Epoch: 2
Cost time: 00h:00m:28.66s
Steps: 893 | Train Loss: 0.0110632 Vali L

In [48]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_IT.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0102  0.1009  0.0600  0.3813
              2         24        0.0102  0.1009  0.0602  0.3814
              1         96        0.0177  0.1330  0.0839  0.5031
              2         96        0.0177  0.1330  0.0834  0.5030
              1         168       0.0192  0.1387  0.0891  0.5248
              2         168       0.0191  0.1383  0.0889  0.5235
MAE           1         24        0.0102  0.1008  0.0572  0.3810
              2         24        0.0104  0.1018  0.0580  0.3849
              1         96        0.0184  0.1356  0.0821  0.5126
              2         96        0.0189  0.1374  0.0826  0.5197
              1         168       0.0205  0.1431  0.0874  0.5416
              2         168       0.0200  0.1415  0.0871  0.5354

In [49]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1345279.750  1159.8619   744.5026  0.0815
              2         24        1345018.875  1159.7495   748.0704  0.0815
              1         96        2733544.000  1653.3433  1080.4557  0.1164
              2         96        2685297.500  1638.6877  1075.3787  0.1153
              1         168       3203129.750  1789.7290  1175.1929  0.1261
              2         168       3249148.250  1802.5394  1180.3934  0.1270
MAE           1         24        1196659.250  1093.9192   676.9175  0.0769
              2         24        1235584.125  1111.5684   693.6528  0.0781
              1         96        2641473.750  1625.2611  1033.8958  0.1144
              2         96        2638311.750  1624.2881  1032.5491  0.1143
              1         168       3125136.250  1767.8055  1126.4341  0.1245
              2         168       3089354.000  1757.6559  1120.9506  0.1238

In [50]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_IT.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_IT.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0103  0.1013  0.0576  0.3830
         MSE            0.0102  0.1009  0.0601  0.3814
96       MAE            0.0186  0.1365  0.0824  0.5162
         MSE            0.0177  0.1330  0.0836  0.5030
168      MAE            0.0203  0.1423  0.0873  0.5385
         MSE            0.0192  0.1385  0.0890  0.5241

In [51]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.216122e+06  1102.7438   685.2851  0.0775
         MSE            1.345149e+06  1159.8057   746.2865  0.0815
96       MAE            2.639893e+06  1624.7746  1033.2224  0.1143
         MSE            2.709421e+06  1646.0155  1077.9172  0.1158
168      MAE            3.107245e+06  1762.7307  1123.6923  0.1242
         MSE            3.226139e+06  1796.1342  1177.7932  0.1265

In [52]:
# Rename folders
new_path_name = 'minmax_IT'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)